# 整理dicom数据

In [52]:
import numpy as np
import os
import re
import pandas as pd
import pydicom

save_path='dataset'
save_name='LabelPathList.csv'

## 后期添加的良性病灶转为dcm

### test

In [53]:
# test
origin_data_path = r'H:\Breast Cancer Project\Data\Add_Data\test\anonymity\benign\49'
stage_list = [os.path.join(origin_data_path,name) for name in os.listdir(origin_data_path)]

In [54]:
stage_list

['H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\1',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\2',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\3',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\4',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\5',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\6',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\7',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\8',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\9']

In [55]:
Image_list = [os.path.join(stage,os.listdir(stage)[0])for stage in stage_list] # 每个期数都会再包一个序列名文件夹

In [56]:
Image_list

['H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\1\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\2\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\3\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\4\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\5\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\6\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\7\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\8\\Series0008_Dyn Ax VIBRANT(1+8)',
 'H:\\Breast Cancer Project\\Data\\Add_Data\\test\\anonymity\\benign\\49\\9\\Series0008_Dyn Ax VIBRANT(1+8)']

In [57]:
Images=[]
slice=0
for i in range(len(Image_list)):
    stage_Images = os.listdir(Image_list[i])
    if i is 0:
        slice = len(stage_Images)
    elif len(stage_Images)!=slice:
        print('slice not match on ',origin_data_path.split('\\')[-4:],' ',i)
        break
    else:
        Images=[os.path.join(Image_list[i],Image)for Image in stage_Images]

In [58]:
# for Image in Images:
file_path=Images[0]
ds = pydicom.dcmread(file_path)
ds.save_as('0001.dcm')

D:\anaconda\envs\Type19\lib\site-packages\pydicom\charset.py:715: UserWarning: Value 'GB18030' cannot be used as code extension, ignoring it
  encodings, py_encodings


In [59]:
ds = pydicom.read_file('0001.dcm')
str=ds[0X0008,0X0032].value
print( ds[0X0008,0X0032].value)

103452


In [65]:
from datetime import datetime

dd = datetime.strftime(datetime.strptime(str,"%H%M%S"), "%H:%M:%S")

In [66]:
print(dd)

10:34:52


In [68]:
# print (endtime - starttime).seconds

# 批量处理

In [9]:
import numpy as np
import os
import re
import pandas as pd
import pydicom

# datas_path = r'H:\Breast Cancer Project\Data\Add_Data\test\anonymity\benign'
# datas_path = r'H:\Breast Cancer Project\Data\Add_Data\baidudouwnload\benign breast lesions1(8)'
# datas_path=r'H:\Breast Cancer Project\Data\Add_Data\baidudouwnload\benign breast lesions2(7)'
datas_path=r'H:\Breast Cancer Project\Data\Add_Data\baiduDoumload(69-108)\data\benign breast lesion'
datas_paths=[os.path.join(datas_path,data) for data in os.listdir(datas_path)]

In [10]:
save_path = r'H:\Breast Cancer Project\Data\New-DCM\Benign'

for origin_data_path in datas_paths:
    patient_name = 'P'+origin_data_path.split('\\')[-1].zfill(3)+'_NOR'
    # print(patient_name)
    dcm_save_path=os.path.join(save_path,patient_name)
    stage_list = [os.path.join(origin_data_path,name) for name in os.listdir(origin_data_path) if not re.search('.nii',name)]
    Image_list = [os.path.join(stage,os.listdir(stage)[0])for stage in stage_list] # 每个期数都会再包一个序列名文件夹
    slice=0

    # 判断数据切片数是否符合
    patient_useful = True
    for i in range(len(Image_list)):
        stage_Images = os.listdir(Image_list[i])
        if i is 0:
            slice = len(stage_Images)
        elif len(stage_Images)!=slice:
            patient_useful=False
            print('slice not match on ',origin_data_path.split('\\')[-4:],' ',i)
            break
    if not patient_useful:
        continue
    print('slice match on ',origin_data_path.split('\\')[-4:])

    for i in range(len(Image_list)):
        stage = np.str(i+1)
        stage_save_path= os.path.join(dcm_save_path,stage)
        # print(stage_save_path)
        Images=[]
        Images_names=[]
        stage_Images = os.listdir(Image_list[i])
        if not os.path.exists(stage_save_path):
            os.makedirs(stage_save_path)
        Images=[os.path.join(Image_list[i],Image)for Image in stage_Images]
        Images_names=['P'+origin_data_path.split('\\')[-1].zfill(3)+'_'+re.findall(r"\d+",string)[0]+'.dcm' for string in os.listdir(Image_list[i])]
        # print(Images)

        for j in range(len(Images)):
            # print(Images[j],' ',Images_names[j])
            ds = pydicom.dcmread(Images[j])
            ds.save_as(os.path.join(stage_save_path,Images_names[j]))

slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '100']


D:\anaconda\envs\Type19\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '101']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '102']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '103']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '104']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '105']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '106']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '107']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '108']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '69']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '70']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '71']
slice match on  ['baiduDoumload(69-108)', 'data', 'benign breast lesion', '72']
slice match on  ['baiduDoumload(